In [41]:
import json
import data_utils
import conceptset_utils
import torch

In [42]:
"""
CLASS_SIM_CUTOFF: Concenpts with cos similarity higher than this to any class will be removed
OTHER_SIM_CUTOFF: Concenpts with cos similarity higher than this to another concept will be removed
MAX_LEN: max number of characters in a concept

PRINT_PROB: what percentage of filtered concepts will be printed
"""

CLASS_SIM_CUTOFF = 0.85
OTHER_SIM_CUTOFF = 0.9
MAX_LEN = 60
PRINT_PROB = 1

dataset = "oct"
device = "cuda"

save_name = "data/concept_sets/{}_filtered_new.txt".format(dataset)

In [43]:
#EDIT these to use the initial concept sets you want

with open("data/concept_sets/gpt3_init/gpt3_{}_important_new.json".format(dataset), "r") as f:
    important_dict = json.load(f)
with open("data/concept_sets/gpt3_init/gpt3_{}_superclass_new.json".format(dataset), "r") as f:
    superclass_dict = json.load(f)
with open("data/concept_sets/gpt3_init/gpt3_{}_around_new.json".format(dataset), "r") as f:
    around_dict = json.load(f)

cls_file = './data/mnist_classes.txt'
with open(cls_file, "r") as f:
    classes = f.read().split("\n")
classes=[   'Choroidal Neovascularization',
            'Diabetic Macular Edema',
            'DRUSEN',
            'normal eyeball']

In [44]:
concepts = set()

for values in important_dict.values():
    concepts.update(set(values))


for values in superclass_dict.values():
    concepts.update(set(values))

for values in around_dict.values():
    concepts.update(set(values))

# with open("data/concept_sets/{}_filtered_new.txt".format(dataset)) as f:
#     concepts = f.read().split("\n")
print(len(concepts))

107


In [45]:
concepts = conceptset_utils.remove_too_long(concepts, MAX_LEN, PRINT_PROB)
print(concepts)

62 can appear in early stages of age-related macular degeneration
70 a series of dark and light bands radiating out from the central circle
66 appear as small, white or yellow spots in the center of the retina
95 a yellowish or white deposit of fatty material and cellular debris on the retina or optic nerve
96 the presence of abnormal blood vessels in the choroid (layer of blood vessels behind the retina)
107 102
['a white sclera', 'located beneath the retina', 'increased reflectivity at the lesion border', 'an iris', 'eye disease', 'a magnifying glass', 'a dark or empty area in the center of vision', 'leakage of fluid and/or blood from the vessels', 'a series of dark and light bands in the middle', 'there are no cysts or lesions', 'an eye chart', 'swelling', 'loss of normal retinal architecture', 'thickening of the central retina', 'A small, yellowish white lesion on the retina', 'the fovea is oval shaped', 'distortion of the central retina', 'a light ring around the edge', 'Disease',

In [46]:
concepts = conceptset_utils.filter_too_similar_to_cls(concepts, classes, CLASS_SIM_CUTOFF, device, PRINT_PROB)

102
102
Class:Diabetic Macular Edema - Concept:diabetic retinopathy, sim:0.866 - Deleting diabetic retinopathy

Class:Diabetic Macular Edema - Concept:macular degeneration, sim:0.853 - Deleting macular degeneration

Class:normal eyeball - Concept:eyeball, sim:0.892 - Deleting eyeball

99


In [47]:
concepts = conceptset_utils.filter_too_similar(concepts, OTHER_SIM_CUTOFF, device, PRINT_PROB)

torch.Size([99, 99])
A common sign of macular degeneration - a sign of early age-related macular degeneration , sim:0.9566 - Deleting A common sign of macular degeneration
Disease - disease , sim:1.0000 - Deleting disease
May be found in other eye diseases - may be seen in other eye diseases , sim:0.9783 - Deleting May be found in other eye diseases
Medical Condition - a medical condition , sim:0.9625 - Deleting Medical Condition
a dark or empty area in the center of vision - a dark or empty spot in the center of vision , sim:0.9719 - Deleting a dark or empty area in the center of vision
a generally oval shape - a round shape , sim:0.9245 - Deleting a generally oval shape
a medical condition - medical condition , sim:0.9625 - Deleting medical condition
a pupil - pupil , sim:0.9223 - Deleting a pupil
a retina - the retina , sim:0.9173 - Deleting a retina
a sign of early age-related macular degeneration - may be a sign of early age-related macular degeneration , sim:0.9721 - Deleting a s

In [48]:
with open(save_name, "w") as f:
    f.write(concepts[0])
    for concept in concepts[1:]:
        f.write("\n" + concept)